# A Novel Approach for Three-Way Classification of Lumbar Spine Degeneration Using Pseudo-Modality Learning to Handle Missing MRI Data

## Libs

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.losses import BinaryCrossentropy
from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import KFold
import joblib
import lightgbm as lgb
import xgboost as xgb
from tqdm import tqdm

## Training Pipeline

In [2]:
attention_embeddings_paths = [
    '/kaggle/input/attention-embeddings-for-rsna/AT2_attention_embeddings_gsl.csv',
    '/kaggle/input/attention-embeddings-for-rsna/AT2_attention_embeddings_hist.csv',
    '/kaggle/input/attention-embeddings-for-rsna/ST1_attention_embeddings_gsl.csv',
    '/kaggle/input/attention-embeddings-for-rsna/ST1_attention_embeddings_hist.csv',
    '/kaggle/input/attention-embeddings-for-rsna/ST2_attention_embeddings_gsl.csv',
    '/kaggle/input/attention-embeddings-for-rsna/ST2_attention_embeddings_hist.csv'    
]

average_embeddings_paths = [
    '/kaggle/input/embeddings-for-rsna/at2-greyscl/final_embeddings.csv',
    '/kaggle/input/embeddings-for-rsna/at2-hist/final_embeddings.csv',
    '/kaggle/input/embeddings-for-rsna/st1-greyscl/final_embeddings.csv',
    '/kaggle/input/embeddings-for-rsna/st1-hist/final_embeddings.csv',
    '/kaggle/input/embeddings-for-rsna/st2-greyscl/final_embeddings.csv',
    '/kaggle/input/embeddings-for-rsna/st2-hist/final_embeddings.csv'
]

medicalnet_embeddings_paths = [
    '/kaggle/input/medicalnet-attention-layers-for-rsna/AT2_attention_embeddings_gsl.csv',
    '/kaggle/input/medicalnet-attention-layers-for-rsna/AT2_attention_embeddings_hist.csv',
    '/kaggle/input/medicalnet-attention-layers-for-rsna/ST1_attention_embeddings_gsl.csv',
    '/kaggle/input/medicalnet-attention-layers-for-rsna/ST1_attention_embeddings_hist.csv',
    '/kaggle/input/medicalnet-attention-layers-for-rsna/ST2_attention_embeddings_gsl.csv',
    '/kaggle/input/medicalnet-attention-layers-for-rsna/ST2_attention_embeddings_hist.csv',
]

labels_paths = [
    '/kaggle/input/preprocessed-dataset/train_data_AT2.csv',
    '/kaggle/input/preprocessed-dataset/train_data_ST1.csv',
    '/kaggle/input/preprocessed-dataset/train_data_ST2.csv'
]

list_of_combination = [
    'AT2 - GSL - Attention Network',
    'AT2 - HIST - Attention Network',
    'ST1 - GSL - Attention Network',
    'ST1 - HIST - Attention Network',
    'ST2 - GSL - Attention Network',
    'ST2 - HIST - Attention Network',
    
    'AT2 - GSL - Average ResNet50',
    'AT2 - HIST - Average ResNet50',
    'ST1 - GSL - Average ResNet50',
    'ST1 - HIST - Average ResNet50',
    'ST2 - GSL - Average ResNet50',
    'ST2 - HIST - Average ResNet50',
    
    'AT2 - GSL - MedicalNet Network',
    'AT2 - HIST - MedicalNet Network',
    'ST1 - GSL - MedicalNet Network',
    'ST1 - HIST - MedicalNet Network',
    'ST2 - GSL - MedicalNet Network',
    'ST2 - HIST - MedicalNet Network'
]

results_df = pd.DataFrame(columns=['Combination', 'Avg_Test_Accuracy', 'Avg_Val_Accuracy'])

all_embedding_paths = [
    *attention_embeddings_paths,
    *average_embeddings_paths,
    *medicalnet_embeddings_paths
]

corresponding_labels_paths = [
    labels_paths[0],  # AT2 - GSL - Attention Network
    labels_paths[0],  # AT2 - HIST - Attention Network
    labels_paths[1],  # ST1 - GSL - Attention Network
    labels_paths[1],  # ST1 - HIST - Attention Network
    labels_paths[2],  # ST2 - GSL - Attention Network
    labels_paths[2],  # ST2 - HIST - Attention Network
    
    labels_paths[0],  # AT2 - GSL - Average ResNet50
    labels_paths[0],  # AT2 - HIST - Average ResNet50
    labels_paths[1],  # ST1 - GSL - Average ResNet50
    labels_paths[1],  # ST1 - HIST - Average ResNet50
    labels_paths[2],  # ST2 - GSL - Average ResNet50
    labels_paths[2],  # ST2 - HIST - Average ResNet50
    
    labels_paths[0],  # AT2 - GSL - MedicalNet Network
    labels_paths[0],  # AT2 - HIST - MedicalNet Network
    labels_paths[1],  # ST1 - GSL - MedicalNet Network
    labels_paths[1],  # ST1 - HIST - MedicalNet Network
    labels_paths[2],  # ST2 - GSL - MedicalNet Network
    labels_paths[2],  # ST2 - HIST - MedicalNet Network
]

In [3]:

def train(embedding_path, labels_path, model_save_path_prefix='xgb_model_col_'):
    embeddings = pd.read_csv(embedding_path)
    labels = pd.read_csv(labels_path)

    id_cols = labels[['study_id', 'series_id']]
    cols_to_impute = labels.drop(columns=['study_id', 'series_id'])
    imputed_cols = cols_to_impute.apply(lambda x: x.fillna(x.mode()[0]))
    labels = pd.concat([id_cols, imputed_cols], axis=1)

    id_cols = labels[['study_id', 'series_id']]
    cols_to_encode = labels.drop(columns=['study_id', 'series_id'])
    encoded_df = cols_to_encode.apply(LabelEncoder().fit_transform)
    final_df = pd.concat([id_cols, encoded_df], axis=1)

    df = pd.merge(embeddings, final_df, on='study_id', how='inner')

    X = df.iloc[:, :512].values
    Y = df.iloc[:, 515:].values

    avg_accuracies = []
    avg_f1_scores = []
    avg_roc_auc_scores = []

    for col in tqdm(range(Y.shape[1]), desc="Training Columns"):
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y[:, col], test_size=0.2, random_state=42)

        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        try:
            minority_class_size = min(np.bincount(Y_train))
            n_neighbors = min(5, minority_class_size - 1)
            smote = SMOTE(random_state=42, k_neighbors=n_neighbors)
            X_train, Y_train = smote.fit_resample(X_train, Y_train)
        except ValueError as e:
            print(f"Skipping SMOTE for column {col} due to error: {e}")

        # Convert the datasets into DMatrix format for XGBoost
        dtrain = xgb.DMatrix(X_train, label=Y_train)
        dtest = xgb.DMatrix(X_test, label=Y_test)

        params = {
            'objective': 'multi:softprob',  # Multi-class classification
            'num_class': len(np.unique(Y_train)),
            'eval_metric': 'mlogloss',
            'eta': 0.1,                      # Learning rate
            'max_depth': 6,                  # Maximum tree depth
            'silent': 1                      # Silent mode
        }

        model = xgb.train(params, dtrain)

        joblib.dump(model, f'{model_save_path_prefix}{col}.joblib')

        Y_pred = model.predict(dtest)
        Y_pred_classes = np.argmax(Y_pred, axis=1)

        accuracy = accuracy_score(Y_test, Y_pred_classes)
        f1 = f1_score(Y_test, Y_pred_classes, average='macro')

        if len(np.unique(Y_test)) > 2:
            roc_auc = roc_auc_score(Y_test, Y_pred, multi_class='ovr')
        else:
            roc_auc = roc_auc_score(Y_test, Y_pred_classes)

        avg_accuracies.append(accuracy)
        avg_f1_scores.append(f1)
        avg_roc_auc_scores.append(roc_auc)

        print(f'Column {col}: Accuracy = {accuracy:.4f}, F1 Score = {f1:.4f}, ROC AUC = {roc_auc:.4f}')

    return np.mean(avg_accuracies), np.mean(avg_f1_scores), np.mean(avg_roc_auc_scores)


In [4]:
results_df = pd.DataFrame(columns=['Combination', 'Avg_Test_Accuracy', 'Avg_F1_Score', 'Avg_ROC_AUC'])

for embedding_path, label_path, name in zip(all_embedding_paths, corresponding_labels_paths, list_of_combination):
    print(f"\nTraining for: {name} - {embedding_path}")
    
    model_save_path_prefix = f'{name}_svm_model_col_'
    
    avg_accuracy, avg_f1_score, avg_roc_auc = train(embedding_path, label_path)
    
    result_row = pd.DataFrame({
        'Combination': [name],
        'Avg_Test_Accuracy': [avg_accuracy],
        'Avg_F1_Score': [avg_f1_score], 
        'Avg_ROC_AUC': [avg_roc_auc],
    })
    
    results_df = pd.concat([results_df, result_row], ignore_index=True)

results_df.to_csv('results_summary.csv', index=False)

print("Training complete. Results saved to 'results_summary.csv'.")


Training for: AT2 - GSL - Attention Network - /kaggle/input/attention-embeddings-for-rsna/AT2_attention_embeddings_gsl.csv


Training Columns:   0%|          | 0/10 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [18:31:00] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
Training Columns:  10%|█         | 1/10 [00:04<00:44,  4.89s/it]

Column 0: Accuracy = 0.9648, F1 Score = 0.7071, ROC AUC = 0.7019


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [18:31:04] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
Training Columns:  20%|██        | 2/10 [00:11<00:48,  6.04s/it]

Column 1: Accuracy = 0.9045, F1 Score = 0.4870, ROC AUC = 0.6237


...

Column 2: Accuracy = 0.5346, F1 Score = 0.3392, ROC AUC = 0.5532


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [18:43:29] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
Training Columns:  80%|████████  | 4/5 [00:17<00:04,  4.43s/it]

Column 3: Accuracy = 0.4388, F1 Score = 0.3103, ROC AUC = 0.5241


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [18:43:34] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
Training Columns: 100%|██████████| 5/5 [00:21<00:00,  4.37s/it]

Column 4: Accuracy = 0.8218, F1 Score = 0.3444, ROC AUC = 0.5432
Training complete. Results saved to 'results_summary.csv'.


In [5]:
results_df

,Combination,Avg_Test_Accuracy,Avg_F1_Score,Avg_ROC_AUC
0,AT2 - GSL - Attention Network,0.795310,0.591667,0.750050
1,AT2 - HIST - Attention Network,0.790117,0.557979,0.784932
2,ST1 - GSL - Attention Network,0.579683,0.371300,0.570229
3,ST1 - HIST - Attention Network,0.588654,0.362911,0.574436
4,ST2 - GSL - Attention Network,0.752128,0.347900,0.566531
5,ST2 - HIST - Attention Network,0.736702,0.344029,0.567584
6,AT2 - GSL - Average ResNet50,0.728811,0.506032,0.713579
7,AT2 - HIST - Average ResNet50,0.728811,0.491778,0.745395
8,ST1 - GSL - Average ResNet50,0.537995,0.359827,0.582732
9,ST1 - HIST - Average ResNet50,0.530607,0.337753,0.547019


In [6]:
def random_predictions(path):
    labels = pd.read_csv(path)

    id_cols = labels[['study_id', 'series_id']]
    cols_to_impute = labels.drop(columns=['study_id', 'series_id'])
    imputed_cols = cols_to_impute.apply(lambda x: x.fillna(x.mode()[0]))
    final_df = pd.concat([id_cols, imputed_cols], axis=1)

    id_cols = final_df[['study_id', 'series_id']]
    cols_to_encode = final_df.drop(columns=['study_id', 'series_id'])
    encoder = OneHotEncoder(sparse_output=False)
    encoded_cols = encoder.fit_transform(cols_to_encode)
    encoded_df = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(cols_to_encode.columns))
    final_df = pd.concat([id_cols, encoded_df], axis=1)

    Y_true = final_df.drop(columns=['study_id', 'series_id']).values

    np.random.seed(42)  # For reproducibility
    random_predictions = np.random.rand(*Y_true.shape)  # Random floats in [0.0, 1.0]

    threshold = np.mean(random_predictions)

    binary_predictions = (random_predictions > threshold).astype(float)

    accuracy = np.mean(np.all(binary_predictions == Y_true, axis=1))

    print(f"Random Predictions Accuracy: {accuracy:.4f}")
    print(f"Self-adjusting Threshold: {threshold:.4f}")

## Random Accuracy

In [7]:
random_predictions('/kaggle/input/preprocessed-dataset/train_data_AT2.csv')
random_predictions('/kaggle/input/preprocessed-dataset/train_data_ST1.csv')
random_predictions('/kaggle/input/preprocessed-dataset/train_data_ST2.csv')

Random Predictions Accuracy: 0.0000
Self-adjusting Threshold: 0.4996
Random Predictions Accuracy: 0.0000
Self-adjusting Threshold: 0.4995
Random Predictions Accuracy: 0.0000
Self-adjusting Threshold: 0.5004
